# Install and Import Dependencies

In [ ]:
! pip install transformers

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# Instantiate Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Encode and Calculate Sentiment

In [5]:
tokens = tokenizer.encode('I hated this, absolutely the worst', return_tensors='pt')

In [8]:
tokens

tensor([[  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 10103, 43060,
           102]])

In [7]:
tokenizer.decode(tokens[0])

'[CLS] i hated this, absolutely the worst [SEP]'

In [10]:
result = model(tokens)

In [13]:
result.logits

tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>)

In [14]:
int(torch.argmax(result.logits))+1

1

# Collect Reviews

In [40]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [41]:
reviews

['Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.',
 'The food and service here was really good. \xa0It was more like tapas food than Mexican food! \xa0The drinks were amazing too!',
 'Visiting from Texas and decided to give this restaurant a try. \xa0We were pleasantly surprised. \xa0While the margaritas are more like martinis, the food was excellent. \xa0More like a tapas, Mexican fusion. \xa0Great way to try different plates.The real treat was Chelsea, our waitress. \xa0Took the time to explain the menu and offer suggestions. \xa0Always smiling and very pleasant. \xa0Best service we have had in Sydney!',
 'Great atmosphere, attentive service, solid margs, and a Tasty menu. The Brisket Tacos were substantial and delicious. The corn ribs??? \xa0Fawgetaboutit! \xa0Unreal. \xa0Wanted to order another plate.',
 "Don't come here expecting 

In [23]:
import pandas as pd
import numpy as np

In [24]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [25]:
df.head()

,review
0,Hotter than I imagine myself to be and firing ...
1,The food and service here was really good. It...
2,Visiting from Texas and decided to give this r...
3,"Great atmosphere, attentive service, solid mar..."
4,Don't come here expecting legit Mexican food b...


In [29]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens[:512])
  return int(torch.argmax(result.logits))+1

In [30]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x))

In [32]:
df

,review,sentiment
0,Hotter than I imagine myself to be and firing ...,5
1,The food and service here was really good. It...,5
2,Visiting from Texas and decided to give this r...,5
3,"Great atmosphere, attentive service, solid mar...",3
4,Don't come here expecting legit Mexican food b...,3
5,Out of all the restaurants that I tried in Syd...,5
6,We came here on a Thursday night @ 5pm and by ...,4
7,The food is fresh and tasty. The scallop cevi...,4
8,Have been here twice and have absolutely loved...,5
9,I was pleasantly surprised at what a great job...,5
